In [135]:

import tensorflow as tf
import numpy as np
import pandas as pd
import ast
from tensorflow.keras import layers, models

# --- 1. The Multi-Rate Sliding Window Generator ---
def combined_generator(e2_csv, srs_csv, e2_len=20, srs_input_rows=100, pred_offset=50):
    # Load E2
    df_e2 = pd.read_csv(e2_csv, header=None).apply(pd.to_numeric, errors='coerce').fillna(0)

    # Load and Parse SRS
    df_srs = pd.read_csv(srs_csv, header=None)
    parsed_srs = []
    for val in df_srs[0]:
        raw_arr = np.array(ast.literal_eval(val), dtype=np.float32)
        arr_reshaped = raw_arr.T.reshape(4, 1536)
        parsed_srs.append(arr_reshaped)
    srs_data = np.stack(parsed_srs, axis=0)

    # We determine steps based on the SRS file because it's the jumping one
    # If SRS jumps by 50, we can only take as many steps as SRS allows
    total_srs_needed_per_step = 100 + pred_offset # 150
    # max_t * 50 + 150 <= total_srs
    num_steps_srs = (len(srs_data) - total_srs_needed_per_step) // 50

    # Also check E2 length
    num_steps_e2 = len(df_e2) - e2_len

    num_steps = min(num_steps_e2, num_steps_srs)

    for t in range(num_steps):
        # --- E2 Input: Slides 1 by 1 ---
        e2_chunk = df_e2.iloc[t : t + e2_len, :19].values
        flat_e2 = np.zeros(5 * 4 * 19, dtype=np.float32)
        flat_e2[:min(e2_chunk.size, 380)] = e2_chunk.flatten()[:380]
        X_e2 = flat_e2.reshape(5, 4, 19)

        # --- SRS Input: Jumps by 50 ---
        srs_start = t * 50
        srs_end = srs_start + srs_input_rows
        X_srs_window = srs_data[srs_start : srs_end] # (100, 4, 1536)
        X_srs = X_srs_window.reshape(20, 20, 1536)

        # --- Label: 50 steps after the SRS window ends ---
        label_idx = srs_end + pred_offset - 1
        y = srs_data[label_idx]

        yield (X_e2, X_srs), y

In [136]:
def get_dataset(e2_list, srs_list, batch_size=4):
    output_signature = (
        (
            tf.TensorSpec(shape=(5, 4, 19), dtype=tf.float32), 
            tf.TensorSpec(shape=(20, 20, 1536), dtype=tf.float32)
        ),
        tf.TensorSpec(shape=(4, 1536), dtype=tf.float32)
    )

    ds = tf.data.Dataset.from_generator(
        lambda: combined_generator(e2_list, srs_list),
        output_signature=output_signature
    )
    
    # NO SHUFFLE - maintains the exact timeline of the CSVs
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [137]:
from tensorflow.keras import layers, models

def create_model(e2_input_shape=(5, 4, 19), srs_input_shape=(20, 20, 1536), lstm_units=128, dropout_rate=0.3):
    
    radio_input = layers.Input(shape=e2_input_shape, name='radio_input')
    
    r1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(radio_input)  # use padded r1
    r1 = layers.BatchNormalization()(r1)
    r1 = layers.Conv2D(32, (2, 2), activation='relu', padding='same')(r1)
    r1 = layers.BatchNormalization()(r1)
    r1 = layers.Dropout(dropout_rate)(r1)
    r1 = layers.ZeroPadding2D(padding=((0,0),(0,1)))(r1)  # pad width by 1

    srs_input = layers.Input(shape=srs_input_shape, name='srs_input')
    s1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(srs_input)
    s1 = layers.Conv2D(1024, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.BatchNormalization()(s1)
    s1 = layers.MaxPooling2D((2, 2))(s1)
    s1 = layers.Conv2D(32, (2, 2), activation='relu', padding='same')(s1)
    s1 = layers.Conv2D(256, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.Conv2D(128, (2, 2), activation='relu', padding='same')(s1)
    s1 = layers.BatchNormalization()(s1)
    s1 = layers.MaxPooling2D((2, 2))(s1)
    s1 = layers.Conv2D(128, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.Conv2D(64, (1, 1), activation='relu', padding='same')(s1)
    s1 = layers.Dropout(dropout_rate)(s1)

    # Concatenate and LSTM
    x = layers.Concatenate(axis=-1)([r1, s1])
    x = layers.Reshape((5*5, 32+64))(x)
    x = layers.LSTM(lstm_units, return_sequences=True)(x)
    x = layers.Flatten()(x)         
    x = layers.Dense(4*1536)(x)      
    output = layers.Reshape((4,1536))(x)

    model = models.Model(inputs=[radio_input, srs_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model


In [138]:
model = create_model()
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 srs_input (InputLayer)         [(None, 20, 20, 153  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_173 (Conv2D)            (None, 20, 20, 64)   884800      ['srs_input[0][0]']              
                                                                                                  
 conv2d_174 (Conv2D)            (None, 20, 20, 1024  66560       ['conv2d_173[0][0]']             
                                )                                                                 
                                                                                           

In [139]:
train_ds = get_dataset("P:/SP Challenge/DataSet/Preprocessed Dataset/E2_1.csv", 
                        "P:/SP Challenge/DataSet/Preprocessed Dataset/pp_srs_1.csv",
                        batch_size=4)
model.fit(train_ds, epochs=5)

Epoch 1/5
13/13 [==============================] - 129s 22ms/step - loss: 6679.6353
Epoch 2/5
13/13 [==============================] - 126s 26ms/step - loss: 6656.2603
Epoch 3/5
13/13 [==============================] - 116s 23ms/step - loss: 6588.8262
Epoch 4/5
13/13 [==============================] - 110s 29ms/step - loss: 6489.9058
Epoch 5/5
13/13 [==============================] - 120s 22ms/step - loss: 6366.3975
